In [7]:
from copy import deepcopy

BOARD_DIRECTIONS = (
    (1,0),
    (1,-1),
    (0,-1),
    (-1,-1),
    (-1,0),
    (-1,1),
    (0,1),
    (1,1),
)

def is_in_board(board, x, y):
    board_width, board_height = get_board_dims(board)
    return 0 <= x < board_width and 0 <= y < board_height

def is_valid_move(board, player, x, y):
    if not is_in_board(board, x, y) or board[y][x] != 0:
        return None
    other_player = get_other_player(player)
    
    board[y][x] = player # Temporarily place tile at x,y
    tiles_to_flip = []
    global BOARD_DIRECTIONS
    for x_delta, y_delta in BOARD_DIRECTIONS:
        x_curr = x + x_delta
        y_curr = y + y_delta
        if not is_in_board(board, x_curr, y_curr):
            continue
        tile_buffer = []
        while board[y_curr][x_curr] == other_player:
            tile_buffer.append((x_curr, y_curr))
            x_curr += x_delta
            y_curr += y_delta
            if not is_in_board(board, x_curr, y_curr):
                break
        if is_in_board(board, x_curr, y_curr) and board[y_curr][x_curr] == player:
            tiles_to_flip.extend(tile_buffer)
    board[y][x] = 0 # Remove temporary tile at x,y
    if len(tiles_to_flip) == 0:
        return None
    return tiles_to_flip

def get_board_dims(board):
    # returns (cols, rows)
    return (len(board[0]), len(board))

def get_valid_moves(board, player):
    cols, rows = get_board_dims(board)
    moves_dict = {}
    for x in range(cols):
        for y in range(rows):
            if board[y][x] == 0:
                tiles_to_flip = is_valid_move(board, player, x, y)
                if tiles_to_flip != None:
                    moves_dict[(x,y)] = tiles_to_flip
    return moves_dict
    
def flip_tiles_on_board(board, player, tiles_to_flip):
    for x,y in tiles_to_flip:
        board[y][x] = player
        
def get_other_player(player):
    if player == 1:
        return 2
    return 1

def evaluate_board(board, player):
    cols, rows = get_board_dims(board)
    other_player = get_other_player(player)
    num_tiles_for_player = 0
    num_tiles_for_other_player = 0
    for x in range(cols):
        for y in range(rows):
            tile = board[y][x]
            if tile == player:
                num_tiles_for_player += 1
            if tile == other_player:
                num_tiles_for_other_player += 1
    return num_tiles_for_player - num_tiles_for_other_player

class Node(object):
    def __init__(self):
        self.children = []
        self.parent = None
        self.eval_val = None
        self.propagated_val = None
        self.move = None # (x, y, player)
        self.board = None
        
def get_num_of_parent_nodes(node):
    depth = 0
    curr_node = node
    while curr_node.parent != None:
        depth += 1
        curr_node = curr_node.parent
    return depth
    
def get_num_of_child_nodes(node):
    depth = 0
    curr_node = node
    while curr_node.children != []:
        depth += 1
        curr_node = curr_node.children[0]
    return depth

def print_board(board):
    cols, rows = get_board_dims(board)
    for y in range(rows):
        line = '|'
        for x in range(cols):
            line += str(board[y][x]) + '|'
        print(line)
    
def build_tree(node, board, player, depth_limit=None, curr_player=None):
    if depth_limit != None and get_num_of_parent_nodes(node) >= depth_limit:
        return
    if curr_player == None:
        curr_player = player
    moves_dict = get_valid_moves(board, curr_player)
    for x,y in moves_dict.keys():
#         print(x,y,curr_player)
        board_copy = deepcopy(board)
#         print(1,board_copy)
        board_copy[y][x] = curr_player # Place player's tile at x,y
#         print(2,board_copy)
#         print('tiles_to_flip', moves_dict[(x,y)])
        flip_tiles_on_board(board_copy, curr_player, moves_dict[(x,y)])
#         print(3,board_copy)
        
        new_node = Node()
        new_node.parent = node
        new_node.eval_val = evaluate_board(board_copy, player)
        new_node.move = (x, y, curr_player)
#         new_node.board = board_copy # For debugging purposes
        node.children.append(new_node)

        build_tree(new_node, board_copy, player, depth_limit, curr_player=get_other_player(curr_player))
        
def get_path_from_parent(node):
    path_to_parent = []
    curr = node
    while curr.parent != None:
        path_to_parent.append(curr.move)
        curr = curr.parent
    return list(reversed(path_to_parent))
    
    
class Lol:
    def __init__(self):
        self.best_score = float("-inf")
        self.best_path = []

#     def minimax(node):
#         best_score = float("-inf")
#         best_path = []
#         _minimax(node, best_score, best_path)
#         return best_score, best_path

    def minimax(self, node):
        if len(node.children) and node.children[0].children == []:
            local_min = float("inf")
            min_node = None
            for child in node.children:
                if child.eval_val < self.best_score: # alpha-beta pruning
                    break
                if child.eval_val < local_min:
                    local_min = child.eval_val
                    min_node = child
            if local_min > self.best_score:
                self.best_score = local_min
                self.best_path = get_path_from_parent(min_node)
        else:
            for child in node.children:
                self.minimax(child)

In [8]:
test_board = [
    [0,0,0],
    [1,2,0],
    [2,1,0],
]
parent_node = Node()
parent_node.board = test_board
build_tree(parent_node, test_board, player=1, depth_limit=4)

In [9]:
haha = Lol()
haha.minimax(parent_node)
print(haha.best_score, haha.best_path)
# get_path_from_parent(parent_node.children[0].children[0].children[0].children[0])

(-2, [(1, 0, 1), (2, 0, 2), (2, 1, 1), (0, 0, 2)])


In [10]:
print(parent_node.children)
print(parent_node.children[0].move)
print(parent_node.children[0].eval_val)
print(parent_node.children[0].board)
print(parent_node.children[0].children[0].move)
print(parent_node.children[0].children[0].eval_val)
print(parent_node.children[0].children[0].board)


[<__main__.Node object at 0x10d94b650>, <__main__.Node object at 0x10d94ba50>]
(1, 0, 1)
3
None
(2, 0, 2)
0
None


In [11]:
test_board = [
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,1,2,0,0,0],
    [0,0,0,2,1,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
]
parent_node = Node()
parent_node.board = test_board
build_tree(parent_node, test_board, player=1, depth_limit=5)

In [12]:
# print(get_num_of_child_nodes(parent_node))

def what_up(node, depth):
    curr_depth = 0
    curr = node
    while curr_depth != depth:
        curr = curr.children[0]
        curr_depth += 1
    print(curr.move)
    print(curr.eval_val)
    print(curr.board)
    print('=================================')

what_up(parent_node, 0)
what_up(parent_node, 1)
what_up(parent_node, 2)
what_up(parent_node, 3)
what_up(parent_node, 4)
what_up(parent_node, 5)





# print(parent_node.children)
# print(parent_node.children[0].move)
# print(parent_node.children[0].eval_val)
# print(parent_node.children[0].board)
# print(parent_node.children[0].children[0].move)
# print(parent_node.children[0].children[0].eval_val)
# print(parent_node.children[0].children[0].board)

None
None
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 2, 0, 0, 0], [0, 0, 0, 2, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]
(4, 2, 1)
3
None
(3, 2, 2)
0
None
(2, 5, 1)
3
None
(5, 4, 2)
0
None
(6, 4, 1)
3
None


In [13]:
import unittest

class ReversiTests(unittest.TestCase):
    
    def test_get_num_of_parent_nodes(self):
        lol = Node()
        self.assertEqual(get_num_of_parent_nodes(lol), 0)
        lol.parent = Node()
        self.assertEqual(get_num_of_parent_nodes(lol), 1)
        lol.parent.parent = Node()
        self.assertEqual(get_num_of_parent_nodes(lol), 2)
    
    def test_build_tree(self):
        test_board = [
            [0,0,0],
            [1,2,0],
            [2,1,0],
        ]
        parent_node = Node()
        build_tree(parent_node, test_board, player=1, depth_limit=2)
        self.assertEqual(parent_node.children[0].eval_val, 3)
        self.assertEqual(parent_node.children[0].move, (1, 0, 1))
        self.assertEqual(parent_node.children[1].move, (2, 1, 1))
        self.assertEqual(parent_node.children[0].children[0].move, (2, 0, 2))
        self.assertEqual(parent_node.children[0].children[0].children, [])
    
    def test_flip_tiles_on_board(self):
        original = [
            [0,1,2],
            [1,1,0],
            [2,1,0],
        ]
        after = [
            [0,1,2],
            [1,2,0],
            [2,1,0],
        ]
        flip_tiles_on_board(original, 2, [(1,1)])
        self.assertEqual(original, after)
    
    def test_get_valid_moves(self):
        test_board = [
            [0,0,0,0,0],
            [0,0,0,0,0],
            [0,0,1,2,0],
            [0,0,2,1,0],
            [0,0,0,0,0]
        ]
        self.assertEqual(get_valid_moves(test_board, 1), {(4, 2): [(3, 2)], (1, 3): [(2, 3)], (3, 1): [(3, 2)], (2, 4): [(2, 3)]})
        test_board_2 = [
            [0,0,0],
            [1,2,0],
            [2,1,0]
        ]
        self.assertEqual(get_valid_moves(test_board, 2), {})
        
    def test_is_valid_move(self):
        test_board = [
            [0,0,0,0,0],
            [0,0,0,0,0],
            [0,0,1,2,0],
            [0,0,2,1,0],
            [0,0,0,0,0]
        ]
        self.assertEqual(is_valid_move(test_board, 1, 3, 1), [(3,2)])
        self.assertEqual(is_valid_move(test_board, 1, 0, 0), None)
        test_board_2 = [
            [0,0,1,0,1],
            [0,0,2,2,0],
            [0,0,0,0,0],
            [0,0,0,0,0],
            [0,0,0,0,0]
        ]
        self.assertEqual(is_valid_move(test_board_2, 1, 2, 2), [(3,1), (2,1)])
        self.assertEqual(is_valid_move(test_board_2, 1, 3, 0), None)
        test_board_3 = [
            [0,0,0],
            [1,2,0],
            [2,1,0],
        ]
        self.assertEqual(is_valid_move(test_board_3, 1, 1, 0), [(1,1)])

    def test_is_in_board(self):
        test_board = [[0]*5]*5
        self.assertEqual(is_in_board(test_board, 4, 4), True)
        self.assertEqual(is_in_board(test_board, 6, 4), False)
        self.assertEqual(is_in_board(test_board, 4, -1), False)

suite = unittest.TestLoader().loadTestsFromTestCase(ReversiTests)
unittest.TextTestRunner(verbosity=2).run(suite)

test_build_tree (__main__.ReversiTests) ... ok
test_flip_tiles_on_board (__main__.ReversiTests) ... ok
test_get_num_of_parent_nodes (__main__.ReversiTests) ... ok
test_get_valid_moves (__main__.ReversiTests) ... ok
test_is_in_board (__main__.ReversiTests) ... ok
test_is_valid_move (__main__.ReversiTests) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.008s

OK


<unittest.runner.TextTestResult run=6 errors=0 failures=0>

In [136]:
def get_cartesian_from_standard(standard):
    return (ord(standard[0]) - 97, int(standard[1]))
get_cartesian_from_standard('f3')

(5, 3)

In [138]:
def get_standard_from_cartesian(cartesian):
    x,y = cartesian
    return str(unichr(x + 97)) + str(y)
get_standard_from_cartesian((5,3))

'f3'

In [302]:
class Laurier:
    def __init__(self):
        self.x = 0
    def change(self, y):
        self.x += y
lol = Laurier()
lol.change(2)
print(lol.x)

2
